## Search space autogeneration

This notebook describes how to generate search space automaticaly from your model.

### Main chapters of this notebook:
1. Setup the environment
1. Prepare dataset and create dataloaders
1. Generate model with search variants and move it into search space
1. Check pretrain, search and tune phases

## Setup the environment
First, let's set up the environment and make some common imports.

In [1]:
import os

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# You may need to uncomment and change this variable to match free GPU index
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import sys

sys.path.append('../')

from pathlib import Path

import torch
import torch.nn as nn

from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_optimizer import RAdam
from torchvision.models.mobilenet import mobilenet_v2

from enot.autogeneration import TransformationParameters
from enot.autogeneration import generate_pruned_search_variants_model
from enot.latency import min_latency
from enot.latency import max_latency
from enot.latency import best_arch_latency
from enot.models import SearchSpaceModel
from enot.optimize import PretrainOptimizer
from enot.optimize import SearchOptimizer

from tutorial_utils.train import accuracy
from tutorial_utils.train import WarmupScheduler

from tutorial_utils.checkpoints import download_autogen_pretrain_checkpoint
from tutorial_utils.checkpoints import download_autogen_search_checkpoint
from tutorial_utils.dataset import create_imagenette_dataloaders
from tutorial_utils.phases import tutorial_pretrain_loop
from tutorial_utils.phases import tutorial_search_loop
from tutorial_utils.phases import tutorial_train_loop

### In the following cell we setup all necessary dirs

* `HOME_DIR` - experiments home directory
* `DATASETS_DIR` - root directory for datasets (imagenette2, ...)
* `PROJECT_DIR` - project directory to save training logs, checkpoints, ...

In [3]:
HOME_DIR = Path.home() / '.optimization_experiments'
DATASETS_DIR = HOME_DIR / 'datasets'
PROJECT_DIR = HOME_DIR / 'search_space_autogeneration'

HOME_DIR.mkdir(exist_ok=True)
DATASETS_DIR.mkdir(exist_ok=True)
PROJECT_DIR.mkdir(exist_ok=True)

## Prepare dataset and create dataloaders

In [4]:
dataloaders = create_imagenette_dataloaders(
    dataset_root_dir=DATASETS_DIR,
    project_dir=PROJECT_DIR,
    input_size=(224, 224),
    batch_size=32,
)

## Generate model with search variants and move it into search space

**IMPORTANT:**<br>
<span style="color:red">**Weights of source model are used for initialization of search variants parameters, so pretrained models are necessary for good results.**</span>

In some cases it is necessary to specify modules that should remain unchanged while generating model.

There are two options:
1. Specify particular module.
1. Specify module type: in this case all modules with specified type remain unchanged.

To do this, pass particular modules or types of modules that need to remain unchaged as parameter in `generate_pruned_search_variant_model`.
__Auto generation supports models with MobileNet-lilke, ResNet-like and EfficientNet-like blocks.__

In [5]:
my_model = mobilenet_v2(pretrained=True)

classifier = my_model.classifier[1]
my_model.classifier = nn.Linear(
    in_features=classifier.in_features,
    out_features=10,
    bias=True,
)
my_model.eval()

first_block = my_model.features[0]  # First MobileNet block in model.
generated_model = generate_pruned_search_variants_model(
    my_model,
    search_variant_descriptors=(
        TransformationParameters(width_mult=1.0),
        TransformationParameters(width_mult=0.75),
        TransformationParameters(width_mult=0.5),
        TransformationParameters(width_mult=0.25),
        TransformationParameters(width_mult=0.0),
    ),
    excluded_modules=[first_block],  # Leave first MobileNet block unchanged.
)
# move model to search space
search_space = SearchSpaceModel(generated_model).cuda()

## Check pretrain, search and tune phases

In this tutorial we use the same pretrain/search/train loops as in <span style="color:green;white-space:nowrap">***1. Tutorial -getting started***</span>.

**IMPORTANT:**<br>
We set `N_EPOCHS`= 3 in this example to make tutorial execution faster. This is not enough for good pretrain quality, and you should set `N_EPOCHS`>= 100 if you want to achieve good results.

In [6]:
N_EPOCHS = 3
N_WARMUP_EPOCHS = 1

len_train = len(dataloaders['pretrain_train_dataloader'])

optimizer = SGD(params=search_space.model_parameters(), lr=0.06, momentum=0.9, weight_decay=1e-4)
pretrain_optimizer = PretrainOptimizer(search_space=search_space, optimizer=optimizer)
scheduler = CosineAnnealingLR(optimizer, T_max=len_train * N_EPOCHS, eta_min=1e-8)
scheduler = WarmupScheduler(scheduler, warmup_steps=len_train * N_WARMUP_EPOCHS)
loss_function = nn.CrossEntropyLoss().cuda()

tutorial_pretrain_loop(
    epochs=N_EPOCHS,
    search_space=search_space,
    pretrain_optimizer=pretrain_optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['pretrain_train_dataloader'],
    validation_loader=dataloaders['pretrain_validation_dataloader'],
    scheduler=scheduler,
)

EPOCH #0
train metrics:
  loss: 2.2176924376792098
  accuracy: 22.498522454119744
validation metrics:
  loss: 1.9676591907778094
  accuracy: 31.199596774193548

EPOCH #1
train metrics:
  loss: 1.7299164647751666
  accuracy: 38.27186760435713
validation metrics:
  loss: 1.5475376594451167
  accuracy: 44.506048387096776

EPOCH #2
train metrics:
  loss: 1.5071947071400094
  accuracy: 47.17730496021027
validation metrics:
  loss: 1.4714765789047364
  accuracy: 49.707661290322584



In [7]:
# We pretrained search space for 3 epochs in this example. In this cell, we are downloading
# search space checkpoint, pretrained for 100 epochs (for demonstration purposes).
checkpoint_path = PROJECT_DIR / 'autogen_pretrain_checkpoint.pth'
download_autogen_pretrain_checkpoint(checkpoint_path)

search_space.load_state_dict(
    torch.load(checkpoint_path)['model'],
)

<All keys matched successfully>

In [8]:
optimizer = RAdam(search_space.architecture_parameters(), lr=0.01)
search_optimizer = SearchOptimizer(search_space=search_space, optimizer=optimizer)

tutorial_search_loop(
    epochs=3,
    search_space=search_space,
    search_optimizer=search_optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['search_train_dataloader'],
    validation_loader=dataloaders['search_validation_dataloader'],
    latency_loss_weight=1e-3,
    latency_type='mmac.thop',
    scheduler=None,
)

EPOCH #0
Constant latency = 45.285376
Min, mean and max latencies of search space: 63.90067199999999, 189.95344640000002, 326.28352
train metrics:
  loss: 0.6127585386556964
  accuracy: 85.08064516129032
  arch_probabilities:
[[0.20089108 0.20087154 0.20187883 0.20055097 0.19580762]
 [0.20160313 0.20231332 0.20198369 0.198722   0.19537789]
 [0.20059489 0.20188448 0.20136295 0.19719078 0.19896689]
 [0.20133255 0.20041639 0.20190012 0.20176904 0.19458196]
 [0.20032638 0.19950524 0.20264046 0.20056172 0.19696628]
 [0.20205148 0.20270073 0.20033981 0.19938368 0.19552428]
 [0.19575156 0.20178169 0.20107716 0.20342435 0.1979653 ]
 [0.20020384 0.19985494 0.2008554  0.20161468 0.1974711 ]
 [0.19818392 0.19836472 0.20272982 0.19929095 0.2014305 ]
 [0.19980751 0.20230387 0.19910848 0.20043467 0.19834547]
 [0.19716325 0.20121427 0.20244043 0.2015328  0.19764929]
 [0.19798389 0.19844064 0.20307718 0.19980898 0.20068923]
 [0.19647868 0.20328866 0.20106438 0.2008176  0.19835068]
 [0.20007047 0.20005

In [9]:
checkpoint_path = PROJECT_DIR / 'autogen_search_checkpoint.pth'
download_autogen_search_checkpoint(checkpoint_path)

search_space.load_state_dict(
    torch.load(checkpoint_path)['model'],
)

print()
print('minimum possible latency', min_latency(search_space))
print('maximum possible latency', max_latency(search_space))
print('current best architecture latency', best_arch_latency(search_space))


minimum possible latency 63.90067203600502
maximum possible latency 326.2835230108032
current best architecture latency 129.7010082479248


In [10]:
# get regular model with the best architecture
best_model = search_space.get_network_with_best_arch().cuda()

In [11]:
optimizer = RAdam(best_model.parameters(), lr=1e-3, weight_decay=1e-4)

tutorial_train_loop(
    epochs=5,
    model=best_model,
    optimizer=optimizer,
    metric_function=accuracy,
    loss_function=loss_function,
    train_loader=dataloaders['tune_train_dataloader'],
    validation_loader=dataloaders['tune_validation_dataloader'],
    scheduler=None,
)

EPOCH #0
train metrics:
  loss: 0.36236336519743534
  accuracy: 88.3835697255236
validation metrics:
  loss: 0.33876564657135355
  accuracy: 89.15322580645162

EPOCH #1
train metrics:
  loss: 0.3636938276443076
  accuracy: 88.29343969466838
validation metrics:
  loss: 0.39798398498979004
  accuracy: 87.88306451612904

EPOCH #2
train metrics:
  loss: 0.4040970911053901
  accuracy: 87.04639477019614
validation metrics:
  loss: 0.3932204434588071
  accuracy: 87.6008064516129

EPOCH #3
train metrics:
  loss: 0.391860035473996
  accuracy: 87.33008273510222
validation metrics:
  loss: 0.3816691086357159
  accuracy: 88.29637096774194

EPOCH #4
train metrics:
  loss: 0.40982427628750495
  accuracy: 86.73463355531084
validation metrics:
  loss: 0.40030874194757593
  accuracy: 87.5

